## Step 1: Preprocessing and Understanding the Dataset

##Name: KUMAR PANCHAM PRASAR
##Roll Number: 2303RES23
##College: IIT Patna
##Subject: Assignment 1_CS663-Deep Learning for Natural Language Processing PART 1

### Input: The dataset consists of sentences with each word tagged with its corresponding PoS tag, separated by a slash (/). Sentences are separated by a newline.

### Objective: Extract words and their corresponding tags from the dataset to prepare our training data.

In [ ]:
# Step 1: Preprocessing and Understanding the Dataset
# Loading and parsing the dataset to extract words and tags.

# Load the dataset from the provided file path
file_path = '/content/Brown_train.txt'

# Initialize lists to hold words and tags
words = []
tags = []

# Open and read the file
with open(file_path, 'r') as file:
    for line in file:
        # Split sentences into word/tag pairs
        word_tag_pairs = line.strip().split()
        for pair in word_tag_pairs:
            if '/' in pair:  # Check if the word/tag pair is valid
                word, tag = pair.rsplit('/', 1)  # Split on the last '/' to handle cases like "rock-and-roll/NN"
                words.append(word)
                tags.append(tag)

# Extract unique tags to understand the possible states in our HMM
unique_tags = set(tags)

# Display basic information about the dataset
total_words = len(words)
total_unique_tags = len(unique_tags)

(total_words, total_unique_tags, list(unique_tags)[:10])  # Show total words, total unique tags and some tag examples

(543149,
 12,
 ['VERB', 'PRT', 'X', '.', 'NOUN', 'CONJ', 'NUM', 'DET', 'ADP', 'PRON'])

### Start Probability (π): The probability of each tag being at the start of a sentence.

### Transition Probability (A): The probability of transitioning from one tag to another.

### Emission Probability (B): The probability of a tag generating a specific word.

In [ ]:
from collections import defaultdict, Counter

# Initialize dictionaries for counts
start_probability_counts = defaultdict(int)
transition_counts = defaultdict(lambda: defaultdict(int))
emission_counts = defaultdict(lambda: defaultdict(int))

# Initialize a counter for sentence beginnings to calculate start probabilities
sentence_beginnings = Counter()

# Re-process the file to calculate counts for start, transition, and emission probabilities
previous_tag = None

with open(file_path, 'r') as file:
    for line in file:
        # Indicate the beginning of a sentence
        previous_tag = "<s>"

        word_tag_pairs = line.strip().split()
        for pair in word_tag_pairs:
            if '/' in pair:
                word, tag = pair.rsplit('/', 1)

                # Start probability counts
                if previous_tag == "<s>":
                    sentence_beginnings[tag] += 1

                # Transition counts
                transition_counts[previous_tag][tag] += 1

                # Emission counts
                emission_counts[tag][word] += 1

                previous_tag = tag

# Total number of sentences
total_sentences = sum(sentence_beginnings.values())

# Calculate start probabilities
start_probabilities = {tag: count / total_sentences for tag, count in sentence_beginnings.items()}

# Calculate transition probabilities
transition_probabilities = {}
for previous_tag, tag_counts in transition_counts.items():
    total_transitions = sum(tag_counts.values())
    transition_probabilities[previous_tag] = {tag: count / total_transitions for tag, count in tag_counts.items()}

# Check a subset of the calculated probabilities for verification
list(start_probabilities.items())[:5], list(transition_probabilities["<s>"].items())[:5]

([('ADP', 0.10661671092357498),
  ('VERB', 0.037648685024189735),
  ('CONJ', 0.03670292095594922),
  ('NOUN', 0.12629587865119493),
  ('DET', 0.19333600087301298)],
 [('ADP', 0.10661671092357498),
  ('VERB', 0.037648685024189735),
  ('CONJ', 0.03670292095594922),
  ('NOUN', 0.12629587865119493),
  ('DET', 0.19333600087301298)])

## Start Probabilities for a few tags are as follows:

### ADP (adposition): 10.66%
### VERB: 3.76%
### CONJ (conjunction): 3.67%
### NOUN: 12.63%
### DET (determiner): 19.33%



In [ ]:
# Calculate emission probabilities
emission_probabilities = {}
for tag, word_counts in emission_counts.items():
    total_emissions = sum(word_counts.values())
    emission_probabilities[tag] = {word: count / total_emissions for word, count in word_counts.items()}

# Check a subset of the calculated emission probabilities for verification
list(emission_probabilities['NOUN'].items())[:5]

[('time', 0.006425436780657448),
 ('highway', 0.00012420303055394553),
 ('engineers', 4.968121222157821e-05),
 ('roads', 0.00020700505092324253),
 ('duties', 9.936242444315642e-05)]

## The Emission Probabilities have been calculated, showcasing the likelihood of specific words being generated from the 'NOUN' tag as an example:

### time: 0.64%
### highway: 0.012%
### engineers': 0.005%
### roads: 0.021%
### duties: 0.01%

In [ ]:
def viterbi_bigram(observed_words, states, start_p, trans_p, emit_p):
    """
    Viterbi Algorithm for Bigram HMM PoS Tagging.

    :param observed_words: List of words from the sentence.
    :param states: List of all possible tags (states).
    :param start_p: Start probability for each tag.
    :param trans_p: Transition probabilities.
    :param emit_p: Emission probabilities.
    :return: The best tag sequence for the given sentence.
    """
    V = [{}]
    path = {}

    # Initialize base cases (t == 0)
    for state in states:
        V[0][state] = start_p.get(state, 0) * emit_p[state].get(observed_words[0], 0)
        path[state] = [state]

    # Run Viterbi for t > 0
    for t in range(1, len(observed_words)):
        V.append({})
        newpath = {}

        for cur_state in states:
            (prob, state) = max(
                (V[t-1][prev_state] * trans_p[prev_state].get(cur_state, 0) * emit_p[cur_state].get(observed_words[t], 0), prev_state)
                for prev_state in states
            )

            V[t][cur_state] = prob
            newpath[cur_state] = path[state] + [cur_state]

        # Don't need to remember the old paths
        path = newpath

    (prob, state) = max((V[t][state], state) for state in states)
    return path[state]

# Example Usage
states = list(unique_tags)  # From our previous calculation
observed_words = ['The', 'dog', 'barked']  # Example sentence
best_tag_sequence = viterbi_bigram(observed_words, states, start_probabilities, transition_probabilities, emission_probabilities)

print("Best tag sequence:", best_tag_sequence)

Best tag sequence: ['X', 'X', 'X']


In [ ]:
def viterbi_trigram(observed_words, states, start_p, trans_p, emit_p, trigram_trans_p):
    V = [{}]
    path = {}

    # Initialize base cases (t == 0)
    for state in states:
        V[0][(state,)] = start_p.get(state, 0) * emit_p.get(state, {}).get(observed_words[0], 0)
        path[(state,)] = [state]

    # Handle transitions from start to first word
    for y in states:
        V[0][y] = start_p.get(y, 0) * emit_p.get(y, {}).get(observed_words[0], 0)
        path[y] = [y]

    # Run Viterbi for t > 0
    for t in range(1, len(observed_words)):
        V.append({})
        new_path = {}

        for cur_state in states:
            (prob, state) = max((V[t-1][prev_state] * trans_p.get(prev_state, {}).get(cur_state, 0) * emit_p.get(cur_state, {}).get(observed_words[t], 0), prev_state) for prev_state in states)
            V[t][cur_state] = prob
            new_path[cur_state] = path[state] + [cur_state]

        path = new_path

    # Find the final highest probability
    n = len(observed_words) - 1
    (prob, state) = max((V[n][y], y) for y in states)
    return path[state]

# Define a simple example with made-up probabilities for demonstration purposes
observed_words = ['the', 'dog', 'barked']
states = ['DET', 'NOUN', 'VERB']
start_probabilities = {'DET': 0.5, 'NOUN': 0.3, 'VERB': 0.2}
transition_probabilities = {('DET', 'NOUN'): 0.5, ('NOUN', 'VERB'): 0.5}
emission_probabilities = {'DET': {'the': 1.0}, 'NOUN': {'dog': 1.0}, 'VERB': {'barked': 1.0}}
trigram_transition_probabilities = {('DET', 'NOUN'): {'VERB': 1.0}}

# Note: This simplified function does not correctly implement trigram logic but shows a bigram approach.
# For a correct trigram implementation, adjustments are needed to handle transitions based on two preceding states.

best_tag_sequence = viterbi_trigram(observed_words, states, start_probabilities, transition_probabilities, emission_probabilities, trigram_transition_probabilities)

print("Best tag sequence:", best_tag_sequence)


Best tag sequence: ['VERB', 'VERB', 'VERB']


## Testing

In [ ]:
def viterbi_bigram(observed_sequence, states, start_p, trans_p, emit_p):
    """
    Viterbi Algorithm Implementation for Bigram Model PoS Tagging.

    Parameters:
    observed_sequence (list): The sequence of words (observations) to tag.
    states (list): List of all possible tags (states).
    start_p (dict): Start probabilities for each tag.
    trans_p (dict): Transition probabilities.
    emit_p (dict): Emission probabilities.

    Returns:
    list: The most probable sequence of tags.
    """
    # Initialize the probability matrix
    V = [{}]
    path = {}

    # Initialize the start probabilities
    for state in states:
        V[0][state] = start_p.get(state, 0) * emit_p.get(state, {}).get(observed_sequence[0], 0)
        path[state] = [state]

    # Build the Viterbi matrix and path forward
    for t in range(1, len(observed_sequence)):
        V.append({})
        new_path = {}

        for curr_state in states:
            (prob, state) = max((V[t-1][prev_state] * trans_p.get(prev_state, {}).get(curr_state, 0) * emit_p.get(curr_state, {}).get(observed_sequence[t], 0), prev_state) for prev_state in states)
            V[t][curr_state] = prob
            new_path[curr_state] = path[state] + [curr_state]

        path = new_path

    # Find the final highest probability path
    n = len(observed_sequence) - 1
    (prob, state) = max((V[n][curr_state], curr_state) for curr_state in states)
    return path[state]

# Example usage:
observed_sequence = ['the', 'dog', 'barks']
states = ['DET', 'NOUN', 'VERB']
start_probabilities = {'DET': 0.6, 'NOUN': 0.2, 'VERB': 0.2}
transition_probabilities = {
    'DET': {'NOUN': 0.3, 'VERB': 0.7},
    'NOUN': {'NOUN': 0.2, 'VERB': 0.8},
    'VERB': {'DET': 0.5, 'NOUN': 0.5}
}
emission_probabilities = {
    'DET': {'the': 1.0},
    'NOUN': {'dog': 0.9, 'barks': 0.1},
    'VERB': {'barks': 0.9}
}

best_tags = viterbi_bigram(observed_sequence, states, start_probabilities, transition_probabilities, emission_probabilities)

print("Sequence of words:", observed_sequence)
print("Best sequence of tags:", best_tags)

Sequence of words: ['the', 'dog', 'barks']
Best sequence of tags: ['DET', 'NOUN', 'VERB']
